In [1]:
import tensorflow as tf
print(tf.__version__)
from keras.utils import np_utils
import matplotlib.pyplot as plt

2.1.0


Using TensorFlow backend.


In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist

# load the training and test data    
(tr_x, tr_y), (te_x, te_y) = fashion_mnist.load_data()

# reshape the feature data
tr_x = tr_x.reshape(tr_x.shape[0], 784)
te_x = te_x.reshape(te_x.shape[0], 784)

# noramlise feature data
tr_x = tr_x / 255.0
te_x = te_x / 255.0

print( "Shape of training features ", tr_x.shape)
print( "Shape of test features ", te_x.shape)


# one hot encode the training labels and get the transpose
tr_y = np_utils.to_categorical(tr_y,10)
tr_y = tr_y.T
print ("Shape of training labels ", tr_y.shape)

# one hot encode the test labels and get the transpose
te_y = np_utils.to_categorical(te_y,10)
te_y = te_y.T
print ("Shape of testing labels ", te_y.shape)

#Reshaping Training data to n x m matrix 
tr_x_reshaped = tr_x.T 
tr_y = tf.constant(tr_y)
tr_x_reshaped = tf.constant(tr_x_reshaped)
#Reshaping test data to n x m matrix 
te_x_reshaped = te_x.T
te_y = tf.constant(te_y)
te_x_reshaped = tf.constant(te_x_reshaped)
print( "Shape of reshaped training features ", tr_x_reshaped.shape)
print( "Shape of reshaped test features ", te_x_reshaped.shape)


Shape of training features  (60000, 784)
Shape of test features  (10000, 784)
Shape of training labels  (10, 60000)
Shape of testing labels  (10, 10000)
Shape of reshaped training features  (784, 60000)
Shape of reshaped test features  (784, 10000)


### Network Architecture A

In [3]:

W1 = tf.Variable(tf.random.normal([300,tr_x_reshaped.shape[0]], mean=0.0, stddev=0.05))
b1 = tf.Variable(tf.zeros(shape = (300,1)))

W2 = tf.Variable(tf.random.normal([10,300], mean=0.0, stddev=0.05))
b2 = tf.Variable(tf.zeros(shape = (10,1)))

def forward_pass(X,W1,W2,b1,b2):
    #Adding Relu Layer
    A1 = tf.add(tf.matmul(tf.cast(W1, tf.float32),tf.cast(X, tf.float32)), b1)
    H1 = tf.nn.relu(A1)
    A2 = tf.add(tf.matmul(tf.cast(W2, tf.float32),tf.cast(H1, tf.float32)), b2)
    H2 = softmax(A2)
    H2 = tf.clip_by_value(H2, 1e-10, 1.0)
    return H2

def cross_entropy(H,y):
    loss  = -tf.reduce_sum(tf.math.multiply(y, tf.math.log(H)),0)
    total_loss = tf.reduce_mean(loss)
    return total_loss
    
    
def calculate_accuracy(y_pred, y):
    predictions = tf.math.argmax(y_pred)
    actual_y = tf.math.argmax(y)
    predictions_correct = tf.cast(tf.equal(predictions, actual_y), tf.float32)
    accuracy = tf.reduce_mean(predictions_correct)
    return accuracy

def softmax(A):
    t = tf.math.exp(A)
    sum_of_t = tf.reduce_sum(t, 0)
    sum_of_t = tf.broadcast_to(sum_of_t, t.shape)
    soft_max_output = tf.math.divide(t, sum_of_t)
    return soft_max_output



In [ ]:
num_Iterations = 500
adam_optimizer = tf.keras.optimizers.Adam()
trainingLoss= []
validationLoss= []
trainingAccuracies = []
validationAccuracies = []

for i in range(num_Iterations):
    with tf.GradientTape() as tape:
        y_pred = forward_pass(tr_x_reshaped, W1,W2, b1,b2)
        currentLoss = cross_entropy(y_pred,tr_y)
        gradients = tape.gradient(currentLoss, [W1,W2, b1,b2])
    accuracy = calculate_accuracy(y_pred, tr_y)
    trainingLoss.append(currentLoss)
    trainingAccuracies.append(accuracy)
    # Calculating Test Loss ana accuracies
    y_pred_test = forward_pass(te_x_reshaped, W1,W2, b1,b2)
    currentLoss_test = cross_entropy(y_pred_test,te_y)
    accuracy_test = calculate_accuracy(y_pred_test, te_y)
    validationLoss.append(currentLoss_test)
    validationAccuracies.append(accuracy_test)
    if i%100 == 0:
        print ("Iteration ", i, ": Loss = ",currentLoss.numpy(), "  Acc: ", accuracy.numpy())
    adam_optimizer.apply_gradients(zip(gradients, [W1,W2, b1,b2]))


Iteration  0 : Loss =  2.3610342   Acc:  0.0565


In [ ]:
plt.plot(validationLoss, label="Val Loss")
plt.plot(validationAccuracies, label="Val Acc")
plt.plot(trainingLoss, label="Train Loss")
plt.plot(trainingAccuracies, label="Train Acc")
plt.legend()

plt.show()

### Network Architecture B

In [ ]:
W1 = tf.Variable(tf.random.normal([300,tr_x_reshaped.shape[0]], mean=0.0, stddev=0.05))
b1 = tf.Variable(tf.zeros(shape = (300,1)))
W2 = tf.Variable(tf.random.normal([100,300], mean=0.0, stddev=0.05))
b2 = tf.Variable(tf.zeros(shape = (100,1)))
W3 = tf.Variable(tf.random.normal([10,100], mean=0.0, stddev=0.05))
b3 = tf.Variable(tf.zeros(shape = (10,1)))

In [ ]:
def forward_pass_B(X,W1,W2,W3,b1,b2,b3):
    #Adding Relu Layer
    A1 = tf.add(tf.matmul(tf.cast(W1, tf.float32),tf.cast(X, tf.float32)), b1)
    H1 = tf.nn.relu(A1)
    A2 = tf.add(tf.matmul(tf.cast(W2, tf.float32),tf.cast(H1, tf.float32)), b2)
    H2 = tf.nn.relu(A2)
    A3 = tf.add(tf.matmul(tf.cast(W3, tf.float32),tf.cast(H2, tf.float32)), b3)
    H3 = softmax(A3)
    H3 = tf.clip_by_value(H3, 1e-10, 1.0)
    return H3



In [ ]:
trainingLoss_B= []
validationLoss_B= []
trainingAccuracies_B = []
validationAccuracies_B = []

for i in range(num_Iterations):
    with tf.GradientTape() as tape:
        y_pred = forward_pass_B(tr_x_reshaped, W1,W2,W3,b1,b2,b3)
        currentLoss = cross_entropy(y_pred,tr_y)
        gradients = tape.gradient(currentLoss, [W1,W2,W3, b1,b2,b3])
    accuracy = calculate_accuracy(y_pred, tr_y)
    trainingLoss_B.append(currentLoss)
    trainingAccuracies_B.append(accuracy)
    # Calculating Test Loss ana accuracies
    y_pred_test = forward_pass_B(te_x_reshaped, W1,W2,W3, b1,b2,b3)
    currentLoss_test = cross_entropy(y_pred_test,te_y)
    accuracy_test = calculate_accuracy(y_pred_test, te_y)
    validationLoss_B.append(currentLoss_test)
    validationAccuracies_B.append(accuracy_test)
    if i%100 == 0:
        print ("Iteration ", i, ": Loss = ",currentLoss.numpy(), "  Acc: ", accuracy.numpy())
    adam_optimizer.apply_gradients(zip(gradients, [W1,W2,W3,b1,b2,b3]))

In [ ]:
plt.plot(validationLoss_B, label="Test Loss")
plt.plot(validationAccuracies_B, label="Test Acc")
plt.plot(trainingLoss_B, label="Train Loss")
plt.plot(trainingAccuracies_B, label="Train Acc")
plt.legend()

plt.show()